# Treasure Hunt Beta Analysis

This notebook provides comprehensive analysis tools for treasurehunt_beta experiments, including TensorBoard event conversion to CSV and various plotting utilities.

## Features
- Convert TensorBoard event files to CSV format
- Plot entity comparisons across different experiment conditions
- Generate learning curves and performance metrics
- Create summary reports and statistical analysis
- Support for multiple entities (Reward, Coin, Gem, Bone, etc.)


In [ ]:
# Import required libraries
import os
import csv
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from tensorflow.python.framework.errors_impl import DataLossError
import tensorflow as tf
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


## 1. TensorBoard to CSV Conversion Functions


In [ ]:
def tensorboard_to_separate_csv(event_file, output_dir):
    """
    Convert TensorBoard event file data to separate CSV files for each tag.
    
    Args:
        event_file (str): Path to the TensorBoard event file.
        output_dir (str): Directory where CSV files should be saved.
    """
    tag_data = defaultdict(list)
    
    try:
        for e in tf.compat.v1.train.summary_iterator(event_file):
            try:
                for v in e.summary.value:
                    if v.HasField('simple_value'):
                        tag = v.tag
                        value = v.simple_value
                        step = e.step
                        tag_data[tag].append([step, value])
            except Exception as record_error:
                print(f"Skipped a corrupt record in file: {event_file}")
    except DataLossError:
        print(f"Encountered DataLossError. Possibly due to incomplete writes in file: {event_file}")
        return

    # Save tag data to CSV files
    for tag, data_rows in tag_data.items():
        filename = f"{output_dir}/{tag.replace('/', '_')}_data.csv"
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Step', 'Value'])
            writer.writerows(data_rows)
        print(f"Data for tag '{tag}' has been written to {filename}")


def process_tensorboard_results(parent_dir, output_parent_dir):
    """
    Process all TensorBoard event files in a directory structure.
    
    Args:
        parent_dir (str): Parent directory containing TensorBoard event files.
        output_parent_dir (str): Parent directory where CSV files should be saved.
    """
    for root, dirs, files in os.walk(parent_dir):
        for file in files:
            if "tfevents" in file:
                event_file = os.path.join(root, file)
                relative_path = os.path.relpath(root, parent_dir)
                output_dir = os.path.join(output_parent_dir, relative_path)
                os.makedirs(output_dir, exist_ok=True)

                try:
                    tensorboard_to_separate_csv(event_file, output_dir)
                    print(f"Processed {event_file} -> {output_dir}")
                except Exception as e:
                    print(f"Failed to process {event_file}: {e}")

print("TensorBoard conversion functions defined!")


## 2. Data Processing Utilities


In [ ]:
def exponential_moving_average(data, alpha):
    """
    Calculate the exponential moving average (EMA) of a 1D array.
    
    Args:
        data (array-like): The input data.
        alpha (float): The smoothing factor (0 < alpha <= 1).
    
    Returns:
        numpy.ndarray: The EMA values.
    """
    if not (0 < alpha <= 1):
        raise ValueError("Alpha must be between 0 and 1.")

    ema = [data[0]]
    for i in range(1, len(data)):
        ema.append(alpha * data[i] + (1 - alpha) * ema[-1])
    return np.array(ema)


def rolling_average(data, window_size):
    """
    Calculate the rolling average of a 1D array.
    
    Args:
        data (array-like): The input data.
        window_size (int): The size of the rolling window.
    
    Returns:
        numpy.ndarray: The rolling average values.
    """
    if window_size < 1:
        raise ValueError("Window size must be at least 1.")
    if len(data) < window_size:
        raise ValueError("Data length must be at least equal to the window size.")
    
    weights = np.ones(window_size) / window_size
    return np.convolve(data, weights, mode='valid')


def trim_and_calculate_mean(array_list):
    """
    Trim arrays to the same length and calculate mean.
    
    Args:
        array_list (list): List of arrays to process.
    
    Returns:
        numpy.ndarray: Mean of trimmed arrays.
    """
    if not array_list:
        return np.array([])
    min_length = min(len(arr) for arr in array_list)
    trimmed_arrays = [arr[:min_length] for arr in array_list]
    trimmed_arrays = np.array(trimmed_arrays)
    return trimmed_arrays


def load_entity_data(folders, entity_name, data_dir='res'):
    """
    Load data for a specific entity across multiple experiment folders.
    
    Args:
        folders (list): List of folder names containing experiment data.
        entity_name (str): Name of the entity to load (e.g., 'Reward', 'Coin').
        data_dir (str): Directory containing the processed CSV files.
    
    Returns:
        list: List of numpy arrays containing the data for each folder.
    """
    collective = [[] for _ in range(len(folders))]
    
    for ixs, folder in enumerate(folders):
        parent_dir = os.path.join(data_dir, folder)
        if not os.path.exists(parent_dir):
            print(f"Warning: Directory {parent_dir} does not exist")
            continue
            
        items = os.listdir(parent_dir)
        
        for item in items:
            if entity_name in item:
                if os.path.isdir(os.path.join(parent_dir, item)):
                    # Handle subdirectory case
                    sub_dir = os.path.join(parent_dir, item)
                    files = os.listdir(sub_dir)
                    if files:
                        data = pd.read_csv(os.path.join(sub_dir, files[0]))
                        collective[ixs].append(data['Value'].to_numpy())
                else:
                    # Handle direct file case
                    data = pd.read_csv(os.path.join(parent_dir, item))
                    collective[ixs].append(data['Value'].to_numpy())
    
    # Process and return mean data
    collective_processed = []
    for i in range(len(collective)):
        if collective[i]:  # Check if there's data
            collective_processed.append(np.mean(trim_and_calculate_mean(collective[i]), axis=0))
        else:
            collective_processed.append(np.array([]))
    
    return collective_processed

print("Data processing utilities defined!")


## 3. Example Usage with TensorBoard Events

Let's convert the TensorBoard event files to CSV and analyze the results.


In [ ]:
# Step 1: Convert TensorBoard results to CSV
print("Converting TensorBoard results to CSV...")
parent_dir = '../runs'
output_dir = 'res'
process_tensorboard_results(parent_dir, output_dir)


In [ ]:
# Step 2: Define experiment parameters
folders = [
    '20250915-222233',
    'treasurehunt_with_respawn_20250916-130915',
    'treasurehunt_with_respawn_20250916-131121',
    'treasurehunt_with_respawn_20250916-131609'
]

labels = [
    'Initial Run',
    'Respawn Run 1',
    'Respawn Run 2', 
    'Respawn Run 3'
]

print(f"Analyzing {len(folders)} experiment runs:")
for i, (folder, label) in enumerate(zip(folders, labels)):
    print(f"  {i+1}. {label} ({folder})")


## 4. Plotting Functions


In [ ]:
def plot_entity_comparison(folders, entity_name, labels=None, window_size=100, 
                          data_dir='res', title=None):
    """
    Plot comparison of entity data across different experiment conditions.
    
    Args:
        folders (list): List of folder names containing experiment data.
        entity_name (str): Name of the entity to plot.
        labels (list, optional): Labels for each condition.
        window_size (int): Window size for rolling average.
        data_dir (str): Directory containing the processed CSV files.
        title (str, optional): Title for the plot.
    """
    data = load_entity_data(folders, entity_name, data_dir)
    
    if labels is None:
        labels = [f'Condition {i+1}' for i in range(len(folders))]
    
    if title is None:
        title = f'{entity_name} Performance Comparison'
    
    plt.figure(figsize=(12, 6))
    colors = plt.cm.Set3(np.linspace(0, 1, len(folders)))
    
    for ixs, d in enumerate(data):
        if len(d) > 0:
            smoothed_data = rolling_average(d, window_size)
            plt.plot(smoothed_data, label=labels[ixs], alpha=0.8, color=colors[ixs], linewidth=2)
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title(title, fontsize=16, fontweight='bold')
    plt.ylabel(f'{entity_name} Value', fontsize=12)
    plt.xlabel('Epoch', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_multiple_entities(folders, entities, labels=None, window_size=100, 
                          data_dir='res'):
    """
    Plot multiple entities in subplots.
    
    Args:
        folders (list): List of folder names containing experiment data.
        entities (list): List of entity names to plot.
        labels (list, optional): Labels for each condition.
        window_size (int): Window size for rolling average.
        data_dir (str): Directory containing the processed CSV files.
    """
    n_entities = len(entities)
    n_cols = min(3, n_entities)
    n_rows = (n_entities + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    if n_entities == 1:
        axes = [axes]
    elif n_rows == 1:
        axes = axes.reshape(1, -1)
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(folders)))
    
    for i, entity in enumerate(entities):
        row = i // n_cols
        col = i % n_cols
        ax = axes[row, col] if n_rows > 1 else axes[col]
        
        data = load_entity_data(folders, entity, data_dir)
        
        for j, d in enumerate(data):
            if len(d) > 0:
                smoothed_data = rolling_average(d, window_size)
                ax.plot(smoothed_data, label=labels[j] if labels else f'Condition {j+1}', 
                       alpha=0.8, color=colors[j], linewidth=2)
        
        ax.set_title(f'{entity} Performance', fontweight='bold')
        ax.set_ylabel(f'{entity} Value')
        ax.set_xlabel('Epoch')
        ax.grid(True, alpha=0.3)
        ax.legend()
    
    # Hide empty subplots
    for i in range(n_entities, n_rows * n_cols):
        row = i // n_cols
        col = i % n_cols
        ax = axes[row, col] if n_rows > 1 else axes[col]
        ax.set_visible(False)
    
    plt.suptitle('Treasure Hunt Performance Metrics', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("Plotting functions defined!")


## 5. Analysis Examples

Now let's analyze the treasure hunt experiment results!


In [ ]:
# Example 1: Plot individual entity comparisons
entities_to_plot = ['Reward', 'Coin', 'Gem', 'Bone']

for entity in entities_to_plot:
    try:
        print(f"Plotting {entity}...")
        plot_entity_comparison(folders, entity, labels, window_size=50, 
                             title=f'{entity} Performance Comparison')
    except Exception as e:
        print(f"Could not plot {entity}: {e}")


In [ ]:
# Example 2: Plot multiple entities in subplots
print("Creating multi-entity comparison plot...")
try:
    plot_multiple_entities(folders, entities_to_plot, labels, window_size=50)
except Exception as e:
    print(f"Could not create multi-entity plot: {e}")


In [ ]:
# Example 3: Load and analyze specific entity data
print("Loading Reward data for detailed analysis...")
reward_data = load_entity_data(folders, 'Reward')

print("\nReward Statistics:")
print("-" * 50)
for i, (folder, data) in enumerate(zip(folders, reward_data)):
    if len(data) > 0:
        print(f"{labels[i]} ({folder}):")
        print(f"  - Final reward: {data[-1]:.4f}")
        print(f"  - Max reward: {data.max():.4f}")
        print(f"  - Min reward: {data.min():.4f}")
        print(f"  - Mean reward: {data.mean():.4f}")
        print(f"  - Std reward: {data.std():.4f}")
        print(f"  - Data points: {len(data)}")
    else:
        print(f"{labels[i]} ({folder}): No data available")
    print()


In [ ]:
# Example 4: Compare different smoothing methods
print("Comparing different smoothing methods for Reward data...")

# Load reward data for the first available run
reward_data = load_entity_data(folders, 'Reward')
if len(reward_data) > 0 and len(reward_data[0]) > 0:
    data = reward_data[0]  # Use first run's data
    
    plt.figure(figsize=(15, 5))
    
    # Original data
    plt.subplot(1, 3, 1)
    plt.plot(data[:1000], alpha=0.7, label='Original', color='blue')
    plt.title('Original Data (first 1000 points)')
    plt.xlabel('Epoch')
    plt.ylabel('Reward')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Rolling average
    plt.subplot(1, 3, 2)
    window_size = 50
    smoothed_rolling = rolling_average(data, window_size)
    plt.plot(smoothed_rolling[:1000], label=f'Rolling Average (window={window_size})', color='red')
    plt.title('Rolling Average')
    plt.xlabel('Epoch')
    plt.ylabel('Reward')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Exponential moving average
    plt.subplot(1, 3, 3)
    alpha = 0.1
    smoothed_ema = exponential_moving_average(data, alpha)
    plt.plot(smoothed_ema[:1000], label=f'EMA (α={alpha})', color='green')
    plt.title('Exponential Moving Average')
    plt.xlabel('Epoch')
    plt.ylabel('Reward')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No reward data available for comparison")


## 6. Summary Report Generation


In [ ]:
def generate_summary_report(folders, entities, labels=None, data_dir='res'):
    """
    Generate a summary report of the experiment results.
    
    Args:
        folders (list): List of folder names containing experiment data.
        entities (list): List of entity names to analyze.
        labels (list, optional): Labels for each condition.
        data_dir (str): Directory containing the processed CSV files.
    """
    if labels is None:
        labels = [f'Condition {i+1}' for i in range(len(folders))]
    
    print("=" * 60)
    print("TREASURE HUNT EXPERIMENT ANALYSIS REPORT")
    print("=" * 60)
    print(f"Generated on: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Number of conditions: {len(folders)}")
    print(f"Entities analyzed: {', '.join(entities)}")
    print()
    
    for entity in entities:
        print(f"## {entity} Analysis")
        print("-" * 40)
        data = load_entity_data(folders, entity, data_dir)
        
        for i, (folder, d) in enumerate(zip(folders, data)):
            if len(d) > 0:
                final_value = d[-1]
                max_value = np.max(d)
                min_value = np.min(d)
                mean_value = np.mean(d)
                std_value = np.std(d)
                
                print(f"### {labels[i]} ({folder})")
                print(f"- Final value: {final_value:.4f}")
                print(f"- Maximum value: {max_value:.4f}")
                print(f"- Minimum value: {min_value:.4f}")
                print(f"- Mean value: {mean_value:.4f}")
                print(f"- Standard deviation: {std_value:.4f}")
                print(f"- Data points: {len(d)}")
                print()
            else:
                print(f"### {labels[i]} ({folder})")
                print("- No data available")
                print()

# Generate the summary report
print("Generating summary report...")
generate_summary_report(folders, entities_to_plot, labels)


## 7. Custom Analysis Examples

You can modify these examples or create your own analysis based on your specific needs.


In [ ]:
# Example: Custom analysis - Compare final performance across runs
print("Custom Analysis: Final Performance Comparison")
print("=" * 50)

# Load all entity data
all_data = {}
for entity in entities_to_plot:
    all_data[entity] = load_entity_data(folders, entity)

# Create a comparison table
comparison_data = []
for i, (folder, label) in enumerate(zip(folders, labels)):
    row = {'Run': label, 'Folder': folder}
    for entity in entities_to_plot:
        if len(all_data[entity][i]) > 0:
            row[f'{entity}_Final'] = all_data[entity][i][-1]
            row[f'{entity}_Max'] = all_data[entity][i].max()
        else:
            row[f'{entity}_Final'] = 'N/A'
            row[f'{entity}_Max'] = 'N/A'
    comparison_data.append(row)

# Display as a table
df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))


## 8. Notes and Tips

- **Data Directory**: The analysis assumes TensorBoard event files are in `../runs` and CSV files are saved to `res/`
- **Window Size**: Adjust the `window_size` parameter for different smoothing levels (smaller = more detail, larger = smoother)
- **Entities**: Modify the `entities_to_plot` list to analyze different metrics
- **Labels**: Update the `labels` list to match your experiment conditions
- **Error Handling**: The code includes try-catch blocks to handle missing data gracefully

### Common Issues:
- If you get "No data available" errors, check that the CSV files were created correctly
- If plots are empty, verify that the entity names match those in your TensorBoard logs
- If smoothing looks too aggressive, reduce the `window_size` parameter
